In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import torchaudio
from sklearn.model_selection import train_test_split
import os
import sys

In [2]:
train_df = pd.read_csv('data/train_dm.csv' , header='infer')
train_df.head(5)

,file,label,path
0,daningram_15,dementia,data/dementia/Dan Ingram/daningram_15.wav
1,terryjones_5,dementia,data/dementia/Terry Jones/terryjones_5.wav
2,maureenforrester_5,dementia,data/dementia/Maureen Forrester/maureenforrest...
3,aileenhernandez_0,dementia,data/dementia/Aileen Hernandez/aileenhernandez...
4,aileenhernandez_5_1,dementia,data/dementia/Aileen Hernandez/aileenhernandez...


In [3]:
valid_df = pd.read_csv('data/valid_dm.csv' , header='infer')
valid_df.head(5)

,file,label,path
0,JimmyCalderwood_5,dementia,data/dementia/Jimmy Calderwood/JimmyCalderwood...
1,vivnicholson_5,dementia,data/dementia/Viv Nicholson/vivnicholson_5.wav
2,IanHolm_2,dementia,data/dementia/Ian Holm/IanHolm_2.wav
3,CharmianCarr_15,dementia,data/dementia/Charmian Carr/CharmianCarr_15.wav
4,CharmianCarr_5,dementia,data/dementia/Charmian Carr/CharmianCarr_5.wav


In [4]:
import assemblyai as aai

# Replace with your API key
aai.settings.api_key = "687063c7417345c4b8de68a676b60714"
transcriber = aai.Transcriber()

In [5]:
def transcribe_audio(FILE_URL):
   transcript = transcriber.transcribe(FILE_URL)
   return transcript.text

In [7]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import numpy as np

# Define the function for transcription
def transcribe_audio(FILE_URL):
    try:
        transcript = transcriber.transcribe(FILE_URL)
        return transcript.text
    except Exception as e:
        # Log the error if needed, and return NaN
        print(f"Error transcribing {FILE_URL}: {e}")
        return np.nan

# Function to handle transcription with threading and progress
def parallel_transcribe(df, column_name, num_threads=4):
    transcriptions = [None] * len(df)
    
    def worker(index, file_url):
        transcriptions[index] = transcribe_audio(file_url)
    
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(worker, idx, url) for idx, url in enumerate(df[column_name])]
        for future in tqdm(as_completed(futures), total=len(futures), desc="Transcribing"):
            future.result()  # This will raise exceptions if any occurred in the threads
    
    return transcriptions


# Apply parallel transcription with progress tracking
train_df['transcription'] = parallel_transcribe(train_df, 'path', num_threads=4)
valid_df['transcription'] = parallel_transcribe(valid_df, 'path', num_threads=4)

# Save the results to CSV
train_df.to_csv('data/train_dm_transcription.csv', index=False)
valid_df.to_csv('data/valid_dm_transcription.csv', index=False)


Transcribing:   0%|          | 0/227 [00:00<?, ?it/s]

Transcribing:  33%|███▎      | 74/227 [19:47<43:09, 16.92s/it]  

Error transcribing data/dementia/Ronan O'Rahilly/ronano'rahilly_15_2.wav: [Errno 2] No such file or directory: "data/dementia/Ronan O'Rahilly/ronano'rahilly_15_2.wav"
Error transcribing data/dementia/Ronan O'Rahilly/ronano'rahilly_15_1.wav: [Errno 2] No such file or directory: "data/dementia/Ronan O'Rahilly/ronano'rahilly_15_1.wav"


Transcribing:  48%|████▊     | 110/227 [28:37<23:09, 11.87s/it]  

Error transcribing data/nodementia/Yoko Ono/YokoOno_2.wav: [Errno 2] No such file or directory: 'data/nodementia/Yoko Ono/YokoOno_2.wav'


...

Transcribing:  60%|██████    | 29/48 [06:28<00:48,  2.53s/it]

Error transcribing data/nodementia/Morgen Freeman/Morgen Freeman_3.wav: [Errno 2] No such file or directory: 'data/nodementia/Morgen Freeman/Morgen Freeman_3.wav'
Error transcribing data/nodementia/Morgen Freeman/Morgen Freeman_2.wav: [Errno 2] No such file or directory: 'data/nodementia/Morgen Freeman/Morgen Freeman_2.wav'
Error transcribing data/nodementia/Donald Sutherland/DonaldSutherland_3.wav: [Errno 2] No such file or directory: 'data/nodementia/Donald Sutherland/DonaldSutherland_3.wav'
Error transcribing data/nodementia/Donald Sutherland/DonaldSutherland_2.wav: [Errno 2] No such file or directory: 'data/nodementia/Donald Sutherland/DonaldSutherland_2.wav'
Error transcribing data/nodementia/Donald Sutherland/DonaldSutherland_1.wav: [Errno 2] No such file or directory: 'data/nodementia/Donald Sutherland/DonaldSutherland_1.wav'


Transcribing:  73%|███████▎  | 35/48 [06:36<00:24,  1.90s/it]

Error transcribing data/nodementia/Dick Cavett/DickCavett_1.wav: [Errno 2] No such file or directory: 'data/nodementia/Dick Cavett/DickCavett_1.wav'
Error transcribing data/nodementia/Dick Cavett/DickCavett_3.wav: [Errno 2] No such file or directory: 'data/nodementia/Dick Cavett/DickCavett_3.wav'
Error transcribing data/nodementia/Queen Elisabeth/Queen Elisabeth_3.wav: [Errno 2] No such file or directory: 'data/nodementia/Queen Elisabeth/Queen Elisabeth_3.wav'
Error transcribing data/nodementia/Anthony Hopkins/AnthonyHopkins_1.wav: [Errno 2] No such file or directory: 'data/nodementia/Anthony Hopkins/AnthonyHopkins_1.wav'
Error transcribing data/nodementia/Anthony Hopkins/AnthonyHopkins_2.wav: [Errno 2] No such file or directory: 'data/nodementia/Anthony Hopkins/AnthonyHopkins_2.wav'
Error transcribing data/nodementia/Anthony Hopkins/AnthonyHopkins_3.wav: [Errno 2] No such file or directory: 'data/nodementia/Anthony Hopkins/AnthonyHopkins_3.wav'
Error transcribing data/nodementia/Charl

Transcribing: 100%|██████████| 48/48 [06:48<00:00,  8.50s/it]
